In [213]:
!python -m tdmclient list
!python -m pip install tk-tools
!python -m tdmclient gui

id:       f3394cac-4181-424e-86bd-f1910fa2563b
group id: 9ebd48b4-772b-47c1-92b0-a43394232bba
name:     Thymio 210
status:   2 (available)
cap:      7
firmware: 13



In [214]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

import time
import math

In [215]:
import numpy as np
from numpy import zeros

In [223]:
# parametres
time_rotation = 7.4
speed_r = 125
speed_f = 125
stop = 0

periode = 4

path = [[0, 150], [150, 150]]

In [217]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    """
    global motor_left_target, motor_right_target
    
    motor_left_target = l_speed
    motor_right_target = r_speed

In [218]:
#input : angle to change the trajectory
@tdmclient.notebook.sync_var
def orientation (angle):
    
    global motor_left_target, motor_right_target
    
    if angle < 0:
        motors(speed_r, stop)
    else :
        motors(stop, speed_r)
        
    time.sleep(time_rotation * abs(angle) / (math.pi))
    motors(stop, stop)

In [219]:
def get_angle_between(vec1, vec2):
    vec1_unit = vec1 / np.linalg.norm(vec1)
    vec2_unit = vec2 / np.linalg.norm(vec2)

    return np.arccos(np.dot(vec1_unit, vec2_unit))

In [220]:
#input : coordinates of two different points 
#return the distance between two points 
def distance (x1, y1, x2, y2):
    dist = np.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))
    return dist

In [221]:
def mouving (path):
    teta = zeros(np.shape(path)[0])
    #teta = diff_angle(path)
    #print (teta)

    goalx = path[0][0]
    goaly = path[0][1]
    
    i = 0
    x_pense = 0
    y_pense = 0
    teta_pense = 0
    
    kx = 0
    ky = 0
    kteta = 0
    #[kx, ky, kteta] = kalman()
    
    #vec1 = [0,0]
    #vec2 = [goalx, goaly]
    #angle = get_angle_between(vec2, vec1)
    #orientation(angle)
    #angle = math.atan2(goaly, goalx)
    #print(angle)

    while i < np.shape(path)[0]-1:
        #[kx, ky, kteta] = kalman()
        teta[i] = math.atan2((goaly - (y_pense + ky)), (goalx - (x_pense + kx))) #change angle according to kalman
        orientation(teta[i] - kteta)
        dist = distance (goalx, goaly, kx, ky)
        if (dist < (speed_f * periode)):
            i = i + 1
            goalx = path[i][0]
            goaly = path[i][1]
            motors(speed_f, speed_f)
            time.sleep(dist/(speed_f/3))
            x_pense = kx + speed_f * periode * np.cos(teta[i-1])
            y_pense = ky + speed_f * periode * np.sin(teta[i-1])
            vec1 = [np.cos(teta[i-1]), np.sin(teta[i-1])]
            vec2 = [goalx - x_pense, goaly - y_pense]
            angle = get_angle_between(vec2, vec1)
            teta_pense = angle + teta[i-1]
            orientation(angle)
        else :
            motors(speed_f, speed_f)
            x_pense = x_pense + kx + speed_f * periode * np.cos(teta[i])
            y_pense = y_pense + ky + speed_f * periode * np.sin(teta[i])
            teta_pense = teta[i]
            time.sleep(periode)
   
    
    # move the robot to the last point
    while x_pense!=path[i][0] and y_pense!=path[i][1]:
        #[kx, ky, kteta] = kalman()
        dist = distance(path[i][0], path[i][1], x_pense + kx, y_pense + ky)
        if (dist < speed_f * periode):
            motors(speed_f, speed_f)
            time.sleep(dist/(speed_f/3))
            x_pense=path[i][0]
            y_pense=path[i][1]
        else :
            motors(speed_f, speed_f)
            x_pense = x_pense + kx + speed_f * periode * np.cos(teta[i])
            y_pense = y_pense + ky + speed_f * periode * np.sin(teta[i])
            time.sleep(periode)
            
    motors(stop, stop)

In [225]:
mouving(path)

1.5707963267948966
1.5707963267948966
x 3.061616997868383e-14
y 500.0
[6.123234e-17 1.000000e+00]
-0.9191450300180579
2.7367008673047097


In [194]:
motors(stop, stop)